In [4]:
import requests
from bs4 import BeautifulSoup

URL = 'https://www.citilink.ru/catalog/ssd-nakopiteli/'
URL = requests.head(URL, allow_redirects=True).url
requests.get(URL)

<Response [200]>

In [5]:
from ipywidgets import IntProgress
from IPython.display import display
import tqdm

def URLsFromPage(base_url = 'https://www.citilink.ru/catalog/ssd-nakopiteli/'):
    
    #функция предназначена для извлечения ссылок на каждый товар в опеределенной категории товаров
    prgBar = IntProgress(min = 0, max = getPagesAmount(requests.get(base_url).text)) # Создаем прогрессбар
    display(prgBar)
    
    links = [] #список для ссылок на каждый товар
    
    for page in range(1, getPagesAmount(requests.get(base_url).text)+1, 1):
        
        URL = base_url + '?p=' + str(page)
        
        html = requests.get(URL).text
    
        products_list = BeautifulSoup(html).findAll('a',
                                        {'class': 'ProductCardVertical__name Link js--Link Link_type_default'})

        for product in products_list:
            links.append('https://www.citilink.ru' + BeautifulSoup.get(product, 'href') + 'properties/')
        prgBar.value += 1
    
    return links
    
    
def getPagesAmount(html1):
    
    pages_amount = BeautifulSoup(html1).find('a', {'class': 'PaginationWidget__page js--PaginationWidget__page PaginationWidget__page_last PaginationWidget__page-link'})

    pages_amount = BeautifulSoup.get(pages_amount, 'data-page')

    pages_amount = int(pages_amount)
    
    return pages_amount

print(0)

0


In [6]:

SSD_LINKS_LIST = URLsFromPage()
    

IntProgress(value=0, max=11)

In [7]:

len(SSD_LINKS_LIST)


485

In [51]:
import requests
import bs4
from bs4 import BeautifulSoup
import re
import time
from pyuser_agent import UA

def getHTML(URL):
    #agent = 'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.3a3pre) Gecko/20100306 Firefox3.6 (.NET CLR 3.5.30729)'
    agent = UA().firefox
    html = requests.get(URL, headers={'User-Agent': agent}).text
    return html

def getSpecifications(html):
    table = BeautifulSoup(html).find('div', {'class': 'col2 xs-col3 ml-col-start-2 ml-col-3'}).findAll('div', {'class': 'Specifications__row'})
    table_2 = {}
    for t in table:
        name = t.find('div', {'class' : 'Specifications__column Specifications__column_name'}).text.strip()
        value = t.find('div', {'class' : 'Specifications__column Specifications__column_value'}).text.strip()
        if re.match('Гарантий', name):
            name = 'Гарантийное обслуживание'
        table_2[name]=value

    return table_2

In [53]:
import requests
import bs4
from bs4 import BeautifulSoup
import re
import time


def getID(HTML):
    try:
        num = BeautifulSoup(HTML).find('div', {'class': 'ProductHeader__product-id'}).text
    except:
        num = BeautifulSoup(HTML).find('div', {'class': 'ProductPageUsersActivity__product-id'}).text
    num = num.strip()
    num2 = re.split(':', num)
    num = num2[1].strip()
    return num


In [61]:
from ipywidgets import IntProgress
from IPython.display import display
import csv
from progress.bar import IncrementalBar
from tqdm import tqdm

#prgBar = IntProgress(min = 0, max = len(SSD_LINKS_LIST)) # Создаем прогрессбар
#display(prgBar)
#bar = IncrementalBar('Countdown', max = len(SSD_LINKS_LIST))

SSD_LIST = []

with open("ssds.csv", mode="w", encoding='cp1251') as w_file:
    for link in tqdm(SSD_LINKS_LIST):
        html = getHTML(link)
        try:
            id_s = getID(html)
        except:
            time.sleep(30)
            html = getHTML(link)
            id_s = getID(html)
        specifications = getSpecifications(html)
        file_writer = csv.writer(w_file, delimiter = ";", lineterminator="\r")
        to_write = {'ID': id_s, **specifications}
        to_write_mas = []
        for k,v in to_write.items():
            to_write_mas.append(k + ':' + v)     
        file_writer.writerow(to_write_mas)
        #prgBar.value += 1
        #bar.next()
        time.sleep(2)
#bar.finish()
print('Program finished!')

 77%|█████████████████████████████████████████████████████████████▋                  | 374/485 [29:55<08:52,  4.80s/it]


AttributeError: 'NoneType' object has no attribute 'text'

In [60]:
with open('html6.txt', 'w', encoding='utf-8') as f:
    f.write(html)

In [45]:
link

'https://www.citilink.ru/product/nakopitel-ssd-kingspec-pci-e-512gb-xf-512-m-2-2280-1742087/properties/'

In [47]:
agent

'Mozilla/5.0 (Android; U; Android; pl; rv:1.9.2.8) Gecko/20100202 Firefox/3.5.8'

In [ ]:
html